# Three bus system

In [3]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import dpsimpy
import re
import math 

# %matplotlib widget

In [4]:
### nominal power
V_nom = 6.9e3


# Simulation parameters
time_step = 0.001
final_time = 1.0

# POWERFLOW FOR INITIALIZATION
time_step_pf = final_time
final_time_pf = final_time + time_step_pf
sim_name_pf = 'Three_bus_system'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# Components
# nodes
n1_pf = dpsimpy.sp.SimNode('n1', dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode('n2', dpsimpy.PhaseType.Single)
n3_pf = dpsimpy.sp.SimNode('n3', dpsimpy.PhaseType.Single)

#injection

# configuration(Synchronous generator 1)
nom_power_G1=100e6
nom_ph_ph_volt_RMS_G1=6.9e3
init_active_power_G1=0.3386e6
init_reactive_power_G1=0.3222e6
set_point_voltage_G1=nom_ph_ph_volt_RMS_G1
t1_ratio=1

#setup
gen1_pf = dpsimpy.sp.ph1.SynchronGenerator("Generator_1", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
# VD means slack bus
gen1_pf.set_parameters(nom_power_G1, nom_ph_ph_volt_RMS_G1, init_active_power_G1,
                       set_point_voltage_G1 * t1_ratio, dpsimpy.PowerflowBusType.VD,init_reactive_power_G1)
gen1_pf.set_base_voltage(V_nom)


# configuration(Synchronous generator 2)
nom_power_G2=100e6
nom_ph_ph_volt_RMS_G2=6.9e3
init_active_power_G2=0.2222e6
init_reactive_power_G2=-0.0285e6
set_point_voltage_G2=nom_ph_ph_volt_RMS_G2
t2_ratio=1

gen2_pf = dpsimpy.sp.ph1.SynchronGenerator("Generator_2", dpsimpy.LogLevel.debug)
#setPointVoltage is defined as the voltage at the transfomer primary side and should be transformed to network side
gen2_pf.set_parameters(nom_power_G2, nom_ph_ph_volt_RMS_G2, init_active_power_G2,
                       set_point_voltage_G2 * t2_ratio, dpsimpy.PowerflowBusType.PV,init_reactive_power_G2)
gen2_pf.set_base_voltage(V_nom)

#line power flow
line_resistance=0.025
pi= math.pi
line_inductance=0.075/(2*pi*50)

line_pf_1 = dpsimpy.sp.ph1.PiLine('PiLine_13', dpsimpy.LogLevel.debug)
line_pf_1.set_parameters(R=line_resistance, L=line_inductance, C=0)
line_pf_1.set_base_voltage(V_nom)

line_pf_2 = dpsimpy.sp.ph1.PiLine('PiLine_31', dpsimpy.LogLevel.debug)
line_pf_2.set_parameters(R=line_resistance, L=line_inductance, C=0)
line_pf_2.set_base_voltage(V_nom)

line_pf_3 = dpsimpy.sp.ph1.PiLine('PiLine_23', dpsimpy.LogLevel.debug)
line_pf_3.set_parameters(R=2*line_resistance, L=2*line_inductance, C=0)
line_pf_3.set_base_voltage(V_nom)


# load power flow on node 3
p_demand=0.5556e6
q_demand=0.2778e6
active_power_L=p_demand
reactive_power_L=q_demand
load_pf = dpsimpy.sp.ph1.Shunt('Load', dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power_L / V_nom**2, - reactive_power_L / V_nom**2)
load_pf.set_base_voltage(V_nom)

'''
# Load on node 3
p_demand=0.5556e6
q_demand=0.2778e6
active_power_L=p_demand
reactive_power_L=q_demand
load_pf = dpsimpy.dp.ph1.RXLoad("Load", dpsimpy.LogLevel.debug)
load_pf.set_parameters(active_power_L, reactive_power_L, V_nom)
'''

# Topology
gen1_pf.connect([n1_pf])
gen2_pf.connect([n2_pf])
line_pf_1.connect([n1_pf, n3_pf])
line_pf_2.connect([n1_pf, n3_pf])
line_pf_3.connect([n2_pf, n3_pf])
load_pf.connect([n3_pf])
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf,n3_pf], [gen1_pf,gen2_pf, line_pf_1,line_pf_2,line_pf_3, load_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('v1', 'v', n1_pf)
logger_pf.log_attribute('v2', 'v', n2_pf)
logger_pf.log_attribute('v3', 'v', n3_pf)
logger_pf.log_attribute('i13', 'i_intf', line_pf_1)
logger_pf.log_attribute('i23', 'i_intf', line_pf_3)


# print
gen1_pf.print_attribute_list()
gen2_pf.print_attribute_list()
line_pf_1.print_attribute_list()
line_pf_2.print_attribute_list()
line_pf_3.print_attribute_list()
load_pf.print_attribute_list()


# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()


Attribute list for object with name Generator_1:
name                          type                size           value                         
-----------------------------------------------------------------------------------------------
P_set                         Real                               338600.000000                 
P_set_pu                      Real                               0.000000                      
Q_set                         Real                               322200.000000                 
Q_set_pu                      Real                               0.000000                      
V_set                         Real                               6900.000000                   
V_set_pu                      Real                               0.000000                      
base_Voltage                  Real                               6900.000000                   
i_intf                        MatrixComplex       1x1            [...]                 

[15:48:47.535015 PiLine_13 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[15:48:47.535030 PiLine_13 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[15:48:47.535441 PiLine_31 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[15:48:47.535447 PiLine_31 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[15:48:47.535890 PiLine_23 warning] Zero value for Capacitance, setting default value of C=1e-12 [F]
[15:48:47.535895 PiLine_23 warning] Zero value for Conductance, setting default value of G=1e-06 [S]
[15:48:47.540426 Three_bus_system info] Initialize simulation: Three_bus_system
[15:48:47.540983 Three_bus_system info] Scheduling tasks.
[15:48:47.541283 Three_bus_system info] Scheduling done.
[15:48:47.541286 Three_bus_system info] Opening interfaces.
[15:48:47.541290 Three_bus_system info] Start synchronization with remotes on 0 interfaces
[15:48:47.541292 Three_bus_system info] Synchron